# Using spatial autocorrelation on CORONAVIRUS
- You need coronapy (see https://github.com/gisfromscratch/corona-py)

In [90]:
from corona_analytics import hopkins_client

corona_client = hopkins_client()
corona_data = corona_client.query_hopkins_data()
corona_data

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20
0,NaN,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,...,94,110,110,120,170,174,237,273,281,299
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,174,186,197,212,223,243,259,277,304,333
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,367,409,454,511,584,716,847,986,1171,1251
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,224,267,308,334,370,376,390,428,439,466
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,4,4,5,7,7,7,8,8,8,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,NaN,Burundi,-3.373100,29.918900,0,0,0,0,0,0,...,0,0,0,0,0,2,2,3,3,3
255,NaN,Sierra Leone,8.460555,-11.779889,0,0,0,0,0,0,...,0,0,0,0,0,1,2,2,2,4
256,"Bonaire, Sint Eustatius and Saba",Netherlands,12.178400,-68.238500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,2,2
257,NaN,Malawi,-13.254308,34.301525,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,3,4


In [91]:
from arcgis.features import FeatureLayer
from arcgis.geometry import Geometry, Point, SpatialReference
from arcgis.geometry.filters import intersects

def query_administrative_layer(latitude, longitude, country_name=""):
    admin_layer = FeatureLayer("https://services.arcgis.com/P3ePLMYs2RVChkJx/ArcGIS/rest/services/World_Administrative_Divisions/FeatureServer/0")
    location = Point()
    location.x = longitude
    location.y = latitude
    location.spatialReference = SpatialReference(4326)
    intersect_filter = intersects(location, sr=location.spatialReference)
    admin_result = admin_layer.query(geometry_filter=intersect_filter)
    
    if 0 == len(admin_result.features):
        return None
    
    if 1 == len(admin_result.features):
        return admin_result.features[0]
    
    if 0 < len(country_name):
        for feature in admin_result.features:
            if country_name.lower() == feature.attributes["COUNTRY"].lower():
                return feature
    
    return None

def geoenrich_data(corona_data):
    geometries = []
    for corona_row_index, corona_row in corona_data.iterrows():
        admin_feature = query_administrative_layer(corona_row["Lat"], corona_row["Long"], corona_row["Country/Region"])
        if None is admin_feature:
            geometries.append(None)
        else:
            geometries.append(admin_feature.geometry)
    return corona_data.assign(geometry = geometries)

In [92]:
corona_data_enriched = geoenrich_data(corona_data)
corona_data_enriched.loc[corona_data_enriched["geometry"].isnull()]

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,geometry
0,NaN,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,...,110,110,120,170,174,237,273,281,299,"{'rings': [[[7270324.91811174, 3948653.4251233..."
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,186,197,212,223,243,259,277,304,333,"{'rings': [[[2283149.56443565, 5055312.9898028..."
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,409,454,511,584,716,847,986,1171,1251,"{'rings': [[[224568.153999359, 3371197.1583084..."
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,267,308,334,370,376,390,428,439,466,"{'rings': [[[172615.325787782, 5231374.5937134..."
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,4,5,7,7,7,8,8,8,10,"{'rings': [[[1848777.16137342, -821128.8012078..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,NaN,Burundi,-3.373100,29.918900,0,0,0,0,0,0,...,0,0,0,0,2,2,3,3,3,"{'rings': [[[3327153.0373716, -340631.44337779..."
255,NaN,Sierra Leone,8.460555,-11.779889,0,0,0,0,0,0,...,0,0,0,0,1,2,2,2,4,"{'rings': [[[-1320156.61523979, 1118889.974846..."
256,"Bonaire, Sint Eustatius and Saba",Netherlands,12.178400,-68.238500,0,0,0,0,0,0,...,0,0,0,0,0,0,2,2,2,"{'rings': [[[-7602534.03085111, 1362876.271423..."
257,NaN,Malawi,-13.254308,34.301525,0,0,0,0,0,0,...,0,0,0,0,0,0,3,3,4,"{'rings': [[[3826346.81386576, -1373406.253493..."
